In [15]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

def display_slices(nifti_path, axis=0, start_slice=None, end_slice=None, step=1, figsize=(15, 15), rows=6, cols=6):
    """
    Display slices from a NIfTI volume
    
    Parameters:
    - nifti_path: path to the .nii.gz file
    - axis: which axis to slice along (0, 1, or 2)
    - start_slice: starting slice number (default None = start from beginning)
    - end_slice: ending slice number (default None = go to end)
    - step: step size between slices (default 1)
    - figsize: size of the figure (width, height)
    - rows: number of rows in the subplot grid
    - cols: number of columns in the subplot grid
    """
    # Load the image
    img = nib.load(nifti_path)
    data = img.get_fdata()
    
    # Get the shape of the volume
    shape = data.shape
    
    # Set default start and end slices if not provided
    if start_slice is None:
        start_slice = 0
    if end_slice is None:
        end_slice = shape[axis]
    
    # Calculate how many slices we'll show
    slices_to_show = list(range(start_slice, end_slice, step))[:rows*cols]
    
    # Create the figure
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.ravel()
    
    # Find the global min and max for consistent windowing
    vmin = np.min(data)
    vmax = np.max(data)
    
    # Display each slice
    for i, slice_idx in enumerate(slices_to_show):
        if i >= rows*cols:
            break
            
        # Extract the slice based on the axis
        if axis == 0:
            slice_data = data[slice_idx, :, :]
        elif axis == 1:
            slice_data = data[:, slice_idx, :]
        else:  # axis == 2
            slice_data = data[:, :, slice_idx]
        
        # Display the slice
        axes[i].imshow(slice_data.T, cmap='gray', vmin=vmin, vmax=vmax)
        axes[i].axis('off')
        axes[i].set_title(f'Slice {slice_idx}')
    
    # Turn off any unused subplots
    for i in range(len(slices_to_show), rows*cols):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Example usage:
# Display first 36 slices along axis 0
# display_slices("path/to/your/file.nii.gz", axis=0)

# Display specific range of slices
# display_slices("path/to/your/file.nii.gz", axis=0, start_slice=50, end_slice=86)

# Display every 5th slice
# display_slices("path/to/your/file.nii.gz", axis=0, step=5)

# Change grid size and figure size
# display_slices("path/to/your/file.nii.gz", axis=0, figsize=(20,20), rows=8, cols=8)

In [9]:
import nibabel as nib
import numpy as np
import os
from scipy.ndimage import rotate

def process_nifti(input_path, output_path, crop_size=(156, 156), final_size=(224, 224), threshold=-500, 
                 roll=30, pitch=30, yaw=30):
    # Load the NIfTI image
    img = nib.load(input_path)
    data = img.get_fdata()
    
    # Only crop dimensions 1 and 2 (keeping all of dim 0)
    cropped_data = data[:, 0:156, 0:156]
    
    # Threshold values below -500
    cropped_data = np.maximum(cropped_data, threshold)
    
    # Calculate padding for dimensions 1 and 2
    dim1_pad = (final_size[0] - crop_size[0]) // 2
    dim2_pad = (final_size[1] - crop_size[1]) // 2
    dim1_pad_after = final_size[0] - crop_size[0] - dim1_pad
    dim2_pad_after = final_size[1] - crop_size[1] - dim2_pad
    
    # Create padding tuple for all 3 dimensions
    pad_widths = ((0, 0),                    # dim 0: no padding
                  (dim1_pad, dim1_pad_after), # dim 1: padding before and after
                  (dim2_pad, dim2_pad_after)) # dim 2: padding before and after
    
    # Pad the cropped data with -500
    padded_data = np.pad(cropped_data, 
                        pad_width=pad_widths,
                        mode='constant', 
                        constant_values=threshold)
    
    # Apply rotations with threshold enforcement after each rotation
    # Roll (rotation around x-axis)
    if roll != 0:
        padded_data = rotate(padded_data, angle=roll, axes=(1, 2), 
                           reshape=False, mode='constant', cval=threshold,
                           order=1)  # Using linear interpolation
        padded_data = np.maximum(padded_data, threshold)  # Re-apply threshold
    
    # Pitch (rotation around y-axis)
    if pitch != 0:
        padded_data = rotate(padded_data, angle=pitch, axes=(0, 2), 
                           reshape=False, mode='constant', cval=threshold,
                           order=1)  # Using linear interpolation
        padded_data = np.maximum(padded_data, threshold)  # Re-apply threshold
    
    # Yaw (rotation around z-axis)
    if yaw != 0:
        padded_data = rotate(padded_data, angle=yaw, axes=(0, 1), 
                           reshape=False, mode='constant', cval=threshold,
                           order=1)  # Using linear interpolation
        padded_data = np.maximum(padded_data, threshold)  # Re-apply threshold
    
    # Final threshold check to be absolutely sure
    padded_data = np.maximum(padded_data, threshold)
    
    # Create a new NIfTI image with the rotated and padded data
    new_img = nib.Nifti1Image(padded_data, img.affine, img.header)
    
    # Update the header to reflect the new dimensions
    new_img.header.set_data_shape(padded_data.shape)
    
    # Save the processed image
    nib.save(new_img, output_path)
    
    # Print the information
    print(f"Original shape: {data.shape}")
    print(f"Final shape: {padded_data.shape}")
    print(f"Applied rotations - Roll: {roll}°, Pitch: {pitch}°, Yaw: {yaw}°")
    print(f"Value range after processing: [{np.min(padded_data)}, {np.max(padded_data)}]")
    
    # Additional verification
    below_threshold = np.sum(padded_data < threshold)
    if below_threshold > 0:
        print(f"WARNING: Found {below_threshold} voxels below threshold!")

def process_directory(input_dir, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Process all .nii.gz files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.nii.gz'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            try:
                process_nifti(input_path, output_path)
                print(f"Successfully processed: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

In [ ]:
process_nifti(r"C:\Users\acer\Desktop\Data\47-4881 L 2014.nii.gz", r"C:\Users\acer\Desktop\Data\47-4881 L 2014_rpy.nii.gz", roll=10, pitch=10, yaw=10)

# This probably based on Euler's angle which isn't the best, check : https://www.youtube.com/watch?v=zjMuIxRvygQ for Quaternions approach

In [12]:
import nibabel as nib
import numpy as np
import os
from random import shuffle, choice
from scipy.ndimage import rotate

def process_without_rotation(input_path, output_path, crop_size=(156, 156), final_size=(224, 224), threshold=-500):
    """Process the image with only cropping and padding, no rotation"""
    img = nib.load(input_path)
    data = img.get_fdata()
    
    # Only crop dimensions 1 and 2 (keeping all of dim 0)
    cropped_data = data[:, 0:156, 0:156]
    
    # Threshold values below -500
    cropped_data = np.maximum(cropped_data, threshold)
    
    # Calculate padding for dimensions 1 and 2
    dim1_pad = (final_size[0] - crop_size[0]) // 2
    dim2_pad = (final_size[1] - crop_size[1]) // 2
    dim1_pad_after = final_size[0] - crop_size[0] - dim1_pad
    dim2_pad_after = final_size[1] - crop_size[1] - dim2_pad
    
    # Create padding tuple for all 3 dimensions
    pad_widths = ((0, 0),                    # dim 0: no padding
                  (dim1_pad, dim1_pad_after), # dim 1: padding before and after
                  (dim2_pad, dim2_pad_after)) # dim 2: padding before and after
    
    # Pad the cropped data with -500
    padded_data = np.pad(cropped_data, 
                        pad_width=pad_widths,
                        mode='constant', 
                        constant_values=threshold)
    
    # Create a new NIfTI image
    new_img = nib.Nifti1Image(padded_data, img.affine, img.header)
    new_img.header.set_data_shape(padded_data.shape)
    nib.save(new_img, output_path)
    
    print(f"Processed original without rotation")
    print(f"Value range: [{np.min(padded_data)}, {np.max(padded_data)}]")

def augment_nifti(input_path, output_path, crop_size=(156, 156), final_size=(224, 224), possible_angles = [10, 15, 20], threshold=-500):
    """Process the image with random rotation augmentation"""
    img = nib.load(input_path)
    data = img.get_fdata()
    
    # Only crop dimensions 1 and 2 (keeping all of dim 0)
    cropped_data = data[:, 0:156, 0:156]
    cropped_data = np.maximum(cropped_data, threshold)
    
    # Calculate padding
    dim1_pad = (final_size[0] - crop_size[0]) // 2
    dim2_pad = (final_size[1] - crop_size[1]) // 2
    dim1_pad_after = final_size[0] - crop_size[0] - dim1_pad
    dim2_pad_after = final_size[1] - crop_size[1] - dim2_pad
    
    pad_widths = ((0, 0),
                  (dim1_pad, dim1_pad_after),
                  (dim2_pad, dim2_pad_after))
    
    padded_data = np.pad(cropped_data, 
                        pad_width=pad_widths,
                        mode='constant', 
                        constant_values=threshold)
    
    rotations = [
        ('roll', (1, 2)),
        ('pitch', (0, 2)),
        ('yaw', (0, 1)),
        ('Skip', 'Skip')
    ]
    shuffle(rotations)

    rotations = rotations[0:3]
    
    # Generate random angles from the discrete set
    angles = {rot[0]: choice(possible_angles) for rot in rotations}
    
    # Apply rotations in random order
    for rot_name, axes in rotations:

        if (rot_name, axes) == ('Skip', 'Skip'):
            continue

        angle = angles[rot_name]
        if angle != 0:  # Skip if angle is 0
            padded_data = rotate(padded_data, angle=angle, axes=axes,
                               reshape=False, mode='constant', cval=threshold,
                               order=1)
            padded_data = np.maximum(padded_data, threshold)
    
    padded_data = np.maximum(padded_data, threshold)
    
    new_img = nib.Nifti1Image(padded_data, img.affine, img.header)
    new_img.header.set_data_shape(padded_data.shape)
    nib.save(new_img, output_path)
    
    print(f"Applied rotations in order:")
    for rot_name, _ in rotations:
        if rot_name == 'Skip':
            print(f"- {rot_name}")
            continue
        print(f"- {rot_name}: {angles[rot_name]}°")
    print(f"Value range: [{np.min(padded_data)}, {np.max(padded_data)}]")

def augment_dataset(input_dir, output_dir, num_augmentations=5):
    """Create multiple augmentations plus one unaugmented version for each image"""
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.endswith('.nii.gz'):
            input_path = os.path.join(input_dir, filename)
            base_name = filename.replace('.nii.gz', '')
            
            # First, create the unaugmented version
            ori_output_path = os.path.join(output_dir, f"{base_name}_ori.nii.gz")
            try:
                process_without_rotation(input_path, ori_output_path)
                print(f"Created unaugmented version for {filename}")
            except Exception as e:
                print(f"Error creating unaugmented version for {filename}: {str(e)}")
            
            # Then create the augmented versions
            for aug_idx in range(num_augmentations):
                output_filename = f"{base_name}_aug{aug_idx}.nii.gz"
                output_path = os.path.join(output_dir, output_filename)
                
                try:
                    augment_nifti(input_path, output_path)
                    print(f"Created augmentation {aug_idx} for {filename}")
                except Exception as e:
                    print(f"Error creating augmentation {aug_idx} for {filename}: {str(e)}")

# Example usage:
# augment_dataset("input_directory", "output_directory", num_augmentations=5)

In [13]:
augment_dataset(r"C:\Users\acer\Desktop\Data", r"C:\Users\acer\Desktop\Data_augmented", num_augmentations=3)

Processed original without rotation
Value range: [-500.0, 1097.9999682977796]
Created unaugmented version for 47-16872 L.nii.gz
Applied rotations in order:
- pitch: 20°
- yaw: 20°
- Skip
Value range: [-500.0, 1072.7358303307055]
Created augmentation 0 for 47-16872 L.nii.gz
Applied rotations in order:
- roll: 10°
- pitch: 15°
- Skip
Value range: [-500.0, 1068.2918982120414]
Created augmentation 1 for 47-16872 L.nii.gz
Applied rotations in order:
- pitch: 15°
- Skip
- yaw: 10°
Value range: [-500.0, 1068.56466826377]
Created augmentation 2 for 47-16872 L.nii.gz
Processed original without rotation
Value range: [-500.0, 1405.000044517219]
Created unaugmented version for 47-16872 R.nii.gz
Applied rotations in order:
- pitch: 15°
- yaw: 10°
- Skip
Value range: [-500.0, 1380.3165104411846]
Created augmentation 0 for 47-16872 R.nii.gz
Applied rotations in order:
- pitch: 10°
- yaw: 10°
- roll: 15°
Value range: [-500.0, 1380.1872020377048]
Created augmentation 1 for 47-16872 R.nii.gz
Applied rot